<a href="https://colab.research.google.com/github/naman065/SecyRecruitmentPCLUB/blob/Computer-Vision/ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computer Vision Task**
Run the code cell by cell!

In [1]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

# *Download the DataSet from GitHub and Upload it on your Google Drive*
PS: Dont change any name while uploading otherwise the program will not work!!!

In [2]:

from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/dataset/train'
validation_dir = '/content/drive/MyDrive/dataset/validation'
test_dir = '/content/drive/MyDrive/dataset/test'

Mounted at /content/drive


**DATA AUGMENTATION**

In [3]:

# Data augmentation and normalization for training
# I have done this to randomize the images and avoid overfitting.
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only normalization for validation and test
val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 288 images belonging to 2 classes.
Found 99 images belonging to 2 classes.
Found 81 images belonging to 2 classes.


**Pre-Trained Model is used and Custom Layers are added for classification into male and female**

In [4]:
#MobileNetV2 is used for this model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)  #Also Done to Avoid OVERFITTING and simplify the MODEL!!!
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

9406464/9406464 [==============================] - 0s 0us/step


**Initial Layers of MobileNetV2 is freezed i.e not used and Model is Trained on the corresponding features obtained from pre trained model**

In [5]:
#Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#I used adam optimizer from a similar code of age detection on kaggle!
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10
9/9 [==============================] - 95s 10s/step - loss: 0.8477 - accuracy: 0.6215 - val_loss: 0.9988 - val_accuracy: 0.5354
Epoch 2/10
9/9 [==============================] - 5s 531ms/step - loss: 0.4967 - accuracy: 0.7812 - val_loss: 0.5364 - val_accuracy: 0.7071
Epoch 3/10
9/9 [==============================] - 5s 553ms/step - loss: 0.4103 - accuracy: 0.8299 - val_loss: 0.5760 - val_accuracy: 0.7273
Epoch 4/10
9/9 [==============================] - 6s 662ms/step - loss: 0.3415 - accuracy: 0.8403 - val_loss: 0.4082 - val_accuracy: 0.8081
Epoch 5/10
9/9 [==============================] - 5s 537ms/step - loss: 0.3101 - accuracy: 0.8785 - val_loss: 0.5323 - val_accuracy: 0.7576
Epoch 6/10
9/9 [==============================] - 6s 725ms/step - loss: 0.2968 - accuracy: 0.8785 - val_loss: 0.4193 - val_accuracy: 0.7879
Epoch 7/10
9/9 [==============================] - 5s 517ms/step - loss: 0.3040 - accuracy: 0.8715 - val_loss: 0.3913 - val_accuracy: 0.8283
Epoch 8/10
9/9 [=====

***THIS WAS WRITTEN TO IMPROVE UPON ACCURACY DONT RUN AS TAKES TIME P:***

In [ ]:
"""
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)"""


**DataSet also includes a test file to test the accuracy of the model**

In [6]:

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')
print(f'Test Loss: {test_loss:.2f}')

3/3 [==============================] - 16s 8s/step - loss: 0.4287 - accuracy: 0.7778
Test Accuracy: 0.78
Test Loss: 0.43


# **Upload the image from your computer provided on GitHub**

In [7]:

from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded_image = files.upload()
img_path = list(uploaded_image.keys())[0]

# Image is pre processed here according to model
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Prediction (male or female)
prediction = model.predict(img_array)
print("Prediction:", "Male" if prediction[0][0] > 0.5 else "Female")


Saving IMG_1931.jpg to IMG_1931.jpg
1/1 [==============================] - 1s 1s/step
Prediction: Male
